In [1]:
# collections：用于高效地创建和管理数据结构
import collections

# pickle：用于对象的序列化和反序列化
import pickle

# torch：PyTorch 库，用于构建和训练深度学习模型
import torch

# numpy：用于高效的数组运算和数学计算
import numpy as np

# pandas：用于数据处理和分析
import pandas as pd

# sklearn.model_selection：用于划分训练集和测试集
from sklearn.model_selection import train_test_split

# matrix_factorization：自定义模块，包含矩阵分解的模型
from matrix_factorization import MF, NCF

# 另一个自定义模块，通常包含实用函数
from utils import *

# sklearn.metrics：用于计算各种评价指标，这里使用的是 AUC（曲线下面积）
from sklearn.metrics import roc_auc_score

# time：用于处理时间相关的任务。
import time

# 设置 NumPy 的随机数生成器的种子，以确保结果的可重现性。
np.random.seed(2020)

# 设置 PyTorch 的随机数生成器的种子，以确保模型初始化和随机抽样的一致性。
torch.manual_seed(2020)

# mse_func：定义一个计算均方误差（MSE）的函数
def mse_func(x, y): return np.mean((x-y)**2)

# 使用 pandas 读取 CSV 文件，并将数据存储在数据框 rdf 中。
# user_id、video_id、play_duration、video_duration、time、date、timestamp、watch_ratio
# 用户ID、视频ID、用户观看时长、视频总时长、观看时间、观看日期、时间戳、观看比例(这个指标可以反映用户对视频内容的兴趣和参与度)
rdf = pd.read_csv("./small_matrix.csv")

# np.array(rdf)：将 pandas 数据框转换为 NumPy 数组
rdf = np.array(rdf)

# 把用户ID、视频ID、观看比例提取出来
rdf = rdf[:, [0, 1, -1]]

# 观看比例标签二值化，如果评分（或标签）小于等于 0.6，则将其设置为 0（代表不喜欢）。如果评分（或标签）大于 0.6，则将其设置为 1（代表喜欢）。
rdf[:, -1][rdf[:, -1] <= 0.6] = 0
rdf[:, -1][rdf[:, -1] > 0.6] = 1

# 随机打乱 rdf 数组中的行。这是为了在训练模型时防止数据的顺序对模型训练产生影响。
np.random.shuffle(rdf)

# 打乱后的数据中选择前 20% 的数据。这通常用于减少计算负担，特别是在处理大型数据集时。
rdf = rdf[:int(0.2 * rdf.shape[0])]

In [2]:
# clean the data
# 对一个多维数组进行排序和数据清理

# rdf按照用户ID进行排序
rdf = rdf[np.argsort(rdf[:, 0])]

# 创建rdf的副本
c = rdf.copy()

# 对用户ID重新编号，用户ID从0开始编号
for i in range(rdf.shape[0]):
    if i == 0:
        c[:, 0][i] = i
        temp = rdf[:, 0][0]
    else:
        if c[:, 0][i] == temp:
            c[:, 0][i] = c[:, 0][i-1]
        else:
            c[:, 0][i] = c[:, 0][i-1] + 1
        temp = rdf[:, 0][i]


# 将c按照视频ID进行排序
c = c[np.argsort(c[:, 1])]

# 创建c的副本
d = c.copy()

# 对视频ID重新编号，视频ID从0开始编号
for i in range(rdf.shape[0]):
    if i == 0:
        d[:, 1][i] = i
        temp = c[:, 1][0]
    else:
        if d[:, 1][i] == temp:
            d[:, 1][i] = d[:, 1][i-1]
        else:
            d[:, 1][i] = d[:, 1][i-1] + 1
        temp = c[:, 1][i]

# 更新rdf
rdf = d.copy()

In [3]:
# 提取rdf的用户ID和视频ID作为特征
x_train = np.array(rdf[:, :2])

# 提取raf的观看比例作为目标值
y_train = np.array(rdf[:, 2])

# 计算用户的数量
num_user = int(np.max(x_train[:, 0]) + 1)

# 计算视频的数量
num_item = int(np.max(x_train[:, 1]) + 1)

# 将x中的数据类型转换为float
x_train = x_train.astype(float)

# 将y中的数据类型转换为float
y_train = y_train.astype(float)

# 将rdf中的数据类型转换为float
rdf = rdf.astype(float)

In [4]:
# 定义sigmoid函数，通常用于二分类
def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

In [5]:
# 定义一个NCF类，NCF类将用户数量和用户ID进行嵌入，然后进行拼接，经过一个全连接层，进行预测。计算均方误差，反向传播
ncf = NCF(num_user, num_item, embedding_k = 64)

# 将类移动到GPU上
ncf.cuda()

# 对ncf进行训练
ncf.fit(x_train, y_train,
       lr=0.01,
       batch_size=8192,
       lamb=1e-5,
       tol=1e-5,
       verbose=True)

# 预测
test_pred, _ = ncf.predict(x_train)

# 平均预测值
print(np.mean(test_pred))

# 最小预测值
print(np.min(test_pred))

# 最大预测值
print(np.max(test_pred))

# 均方误差
mse_ncf = mse_func(y_train, test_pred)

# 曲线下面积
auc_ncf = roc_auc_score(y_train, test_pred)

print(mse_ncf, auc_ncf)


# 预训练ncf模型，生成预测的评分

[NCF] epoch:32, xent:16.14908790588379
0.6442863
0.0032634484
0.9947425
0.14017118144220525 0.8559954337766199


In [6]:
# 定义一个MF类，MF类将用户数量和用户ID进行嵌入，然后计算向量相似度，在计算二元交叉熵损失函数反向传播
mf_pretrain = MF(num_user, num_item, embedding_k = 64)
mf_pretrain.cuda()

# 训练
mf_pretrain.fit(x_train, y_train, 
    lr=0.01,
    batch_size=2048,
    lamb=1e-5,
    tol=1e-5,
    verbose=True)

# 预测
test_pred, _ = mf_pretrain.predict(x_train)
print(np.mean(test_pred))
print(np.min(test_pred))
print(np.max(test_pred))
mse_mf = mse_func(y_train, test_pred)
auc_mf = roc_auc_score(y_train, test_pred)

print(mse_mf, auc_mf)

# 

[MF] epoch:44, xent:169.26480907201767
0.6426461
0.0036583338
0.99271023
0.10468488955644455 0.931555387883102


In [7]:
# 把rdf中用户不喜欢的物品的用户-物品对提取出来作为C类
class_C = np.c_[np.array(rdf[rdf[:, 2] == 0][:, 0]), np.array(rdf[rdf[:, 2] == 0][:, 1])]

In [8]:
# 计算用户喜欢视频的数量
all_num = len(rdf[rdf[:, 2] == 1][:, 0])

# 设置比例
d_e_ratio = 0.5

# temp_ui是第三列等于1的行的所有第一列和第二列，即用户喜欢视频的用户-视频对
temp_ui = np.c_[np.array(rdf[rdf[:, 2] == 1][:, 0]), np.array(rdf[rdf[:, 2] == 1][:, 1])]

# 使用ncf模型进行预测
temp_rating, z_emb = ncf.predict(temp_ui)
# temp_rating为预测评分，z_emb为用户向量和视频向量的拼接

# temp_rdf为预测评分从高到低排序后的数组
temp_rdf = temp_ui[np.argsort(-temp_rating)]

##################################################################
# class_D为temp_rdf中前一半的数据
class_D = temp_rdf[:int(d_e_ratio * len(temp_rating))]

# class_E为temp_rdf中后一半的数据
class_E = temp_rdf[int(d_e_ratio * len(temp_rating)):]
###################################################################

In [9]:
# 设置比例
a_b_ratio = 0.5

# rdf80%的数量
all_num = int(len(rdf[:, 0]) * 0.8)

# 创建一个全零的 DataFrame，大小为 (num_user, num_item)，表示所有用户与物品的组合。stack() 将其转为一维 Series，并用 reset_index() 重置索引，以便获取所有用户-物品对。
all_data = pd.DataFrame(
    np.zeros((num_user, num_item))).stack().reset_index()

# 将 all_data 转换为 NumPy 数组，并提取前两列（用户和物品），得到所有可能的用户-物品对。
all_data = all_data.values[:, :2]

# 通过集合运算，找出在 all_data 中但不在 rdf 中的用户-物品对，形成未标记数据 unlabeled_x。
unlabeled_x = np.array(
    list(set(map(tuple, all_data)) - set(map(tuple, rdf[:, :2]))), dtype=int)

# 打乱未标记的数据
np.random.shuffle(unlabeled_x)

# 选择前all_num的数量
unlabeled_x = unlabeled_x[:all_num]

# 使用ncf模型对未标记的数据进行预测
temp_rating, z_emb = ncf.predict(unlabeled_x)

# 对未标记的数据的预测评分降序排序
temp_rdf = unlabeled_x[np.argsort(-temp_rating)]

##########################################################################
# class_A为前一半
class_A = temp_rdf[:int(a_b_ratio * len(temp_rating))]

# class_B为后一半
class_B = temp_rdf[int(a_b_ratio * len(temp_rating)):]
#########################################################################

In [10]:
# class_new_A代表永不购买者
temp = np.zeros([class_A.shape[0], 4])
class_new_A = np.c_[class_A, temp]

# class_new_B代表不领优惠卷购买者
temp = np.zeros([class_B.shape[0], 4])
temp[:,[2,3]] = int(1)
class_new_B = np.c_[class_B, temp]

# class_new_C代表只领优惠券者
temp = np.zeros([class_C.shape[0], 4])
temp[:,1] = 1
class_new_C = np.c_[class_C, temp]

# class_new_D代表有优惠卷的时候购买
temp = np.zeros([class_D.shape[0], 4])
temp[:,[1, 3]] = 1
class_new_D = np.c_[class_D, temp]

# class_new_E代表始终购买者
temp = np.zeros([class_E.shape[0], 4])
temp[:,[1,2,3]] = 1
class_new_E = np.c_[class_E, temp]

# 合并成完整数据集
constructed_data = np.r_[np.r_[np.r_[np.r_[class_new_A,class_new_B],class_new_C],class_new_D],class_new_E]

In [11]:
# 加入类别数组
z = np.ones(constructed_data.shape[0])
for i in range(len(z)):
    if constructed_data[i][3] == 0 and constructed_data[i][4] == 1 and constructed_data[i][5] == 1:
        z[i] = 2
    elif constructed_data[i][3] == 1 and constructed_data[i][4] == 0 and constructed_data[i][5] == 0:
        z[i] = 3
    elif constructed_data[i][3] == 1 and constructed_data[i][4] == 0 and constructed_data[i][5] == 1:
        z[i] = 4
    elif constructed_data[i][3] == 1 and constructed_data[i][4] == 1 and constructed_data[i][5] == 1:
        z[i] = 5
constructed_data = np.c_[constructed_data, z]

In [12]:
# 随机打乱构造的数组
np.random.shuffle(constructed_data)

# 分割数据集
constructed_data_train = constructed_data[:int(0.7*constructed_data.shape[0])]
constructed_data_test = constructed_data[int(0.7*constructed_data.shape[0]):]

In [13]:
# 调用MF模型进行预测，正负样本
all_pred, z_emb = mf_pretrain.predict(constructed_data_train[:,:2])
# 生成二元随机变量
all_pred_bi = np.random.binomial(1, all_pred)

# 模型预测为正类的样本
T_1 = constructed_data_train[all_pred_bi == 1]

# 模型预测为负类的样本
T_0 = constructed_data_train[all_pred_bi == 0]

T_1 = np.c_[np.c_[np.c_[T_1[:, :2], np.ones(T_1.shape[0])], T_1[:, 3]], T_1[:, 5]]
T_0 = np.c_[np.c_[np.c_[T_0[:, :2], np.zeros(T_0.shape[0])], T_0[:, 2]], T_0[:, 4]]

# 用户特征
x_tr = np.r_[T_0[:, :2], T_1[:, :2]]

# 激励政策
t_tr = np.r_[np.zeros(T_0.shape[0]), np.ones(T_1.shape[0])]

# 积极收集奖励
c_tr = np.r_[T_0[:,3], T_1[:,3]]

# 结果
y_tr = np.r_[T_0[:,4], T_1[:,4]]

In [14]:
file = open("constructed_data", "wb")
pickle.dump(constructed_data_train, file)
pickle.dump(constructed_data_test, file)
pickle.dump(x_tr, file)
pickle.dump(t_tr, file)
pickle.dump(c_tr, file)
pickle.dump(y_tr, file)
file.close()